In [1]:
import requests
import numpy as np
import datetime

# Group Details

Group Number:

Group Member 1:

Group Member 2:

Group Member 3:

Target Name:

Target Period:


Comments:

# Set up your API token (the password that gives access to LCO)

In [2]:
# This is the password for a "PASEA account" on the LCO Network
API_TOKEN = 'XXX'

# This is the proposal ID for the successful proposal that allows us to use the LCO Network of telescopes
# The "PASEA account" has access to this proposal for checking observation paramters
# BUT the "PASEA account" does not have permission to submit observations.
PROPOSAL_ID = 'LCOEPO2021B-001'

# Determine the required cadence of the observations

In [57]:
# Determine when the observations are to take place
# The Jitter is the window within which your observation may take place. 

target_known_period_hours = 5.358

# Want 0.035 phase intervals between each observation (this is true for all targets):
observation_cadence_hours = target_known_period_hours*0.035 
observation_cadence_hours = np.floor(observation_cadence_hours*60)/60
print('The observing cadence is {} mins'.format(observation_cadence_hours*60))

# Want jitter to equal observation_cadence_hours
# to "fully cover" time
jitter = observation_cadence_hours

# Assume UTC. # Assume UTC; You will get exposures between start and end every "observation_cadence_hours"
total_observation_startToEnd = 5.358*1.2
print('If start at 17:30, when end? A:',17.5+total_observation_startToEnd, '.9296 hr in min is: ', .9296*60)

LCO_input_cadence = {
    'start': '2022-08-07 8:30:00',
    'end': '2022-08-07 8:50:00',
    'period': observation_cadence_hours,
    'jitter': jitter
}

The observing cadence is 11.0 mins
If start at 17:30, when end? A: 23.9296 .9296 hr in min is:  55.775999999999996


# Input details on the target

In [58]:
LCO_input_target = {
    'name': 'Target_J',
    'type': 'ICRS',
    'ra': '259.44602',
    'dec': '31.60057',
    'epoch': 2000
}

# Input some other observing details

In [59]:
# Keep Max_airmass at 1.6
# min_lunar_distance is in DEGREES, keep at 20 degrees

LCO_input_constraints = {
    'max_airmass': 2.0,
    'min_lunar_distance': 20
}

In [60]:
# This does not need to be changed.
# Exposure time is in SECONDS

LCO_input_configurations = [
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': LCO_input_target,
        'constraints': LCO_input_constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'v'
                }
            }
        ]
    },
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': LCO_input_target,
        'constraints': LCO_input_constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'b'
                }
            }
        ]
    }
]

In [61]:
# This cell does not need to be edited.

# We do not provide windows for a cadence request
LCO_input_windows = []

# The telescope class that should be used for this observation
LCO_input_location = {
    'telescope_class': '0m4'
}

In [62]:
# Please input the name of your observation request. Eg. Group_Star_Target_B
LCO_input_requestgroup = {
    'name': 'TestExposure_Target_J',  # The title (group name)
    'proposal': PROPOSAL_ID,
    'ipp_value': 1.05,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'cadence': LCO_input_cadence,
        'configurations': LCO_input_configurations,
        'windows': LCO_input_windows,
        'location': LCO_input_location,
    }]
}

# Check your observation request parameters

Below cells allows you to submit your request to LCO to be checked for errors. This does not actually submit the observing to the telescopes. In order to run the below cells, you will need access to the internet to send your requst to LCO to be checked.

In [55]:
# Given a well formed RequestGroup containing a single Request that has a cadence key, 
# this endpoint returns a new RequestGroup with multiple Requests generated by the cadence function. 

response = requests.post(
    'https://observe.lco.global/api/requestgroups/cadence/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=LCO_input_requestgroup  # Make sure you use json!
)

try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

Request failed: b'{"non_field_errors":["The proposal you are trying to submit with is currently inactive"]}'


HTTPError: 400 Client Error: Bad Request for url: https://observe.lco.global/api/requestgroups/cadence/

In [ ]:
# Print all the observation requests that will be submitted below in this notebook
cadence_request = response.json()

print('Cadence generated {} requests'.format(len(cadence_request['requests'])))
i = 1
for request in cadence_request['requests']:
    print('Request {0} window start: {1} window end: {2}'.format(
        i, request['windows'][0]['start'], request['windows'][0]['end']
    ))
    i = i + 1

In [ ]:
# Submit the fully formed RequestGroup to validate it, and see how much observing time it will use. 
response = requests.post(
    'https://observe.lco.global/api/requestgroups/validate/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure the API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('API call failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()  # The API will return the newly submitted requestgroup as json

print('Total number of observing hours needed for this request: ',
      requestgroup_dict['request_durations']['duration']/60./60)

# Actually submit the observation

You will not be able to do this, as the "PASEA account" key does not have permission to submit observations. 

Once you are happy with your request, Direct Message on Slack Dr. Heidi White with the link to this Google Colab noebook. 

You can get the link to the notebook by clicking the share button in the upper right of the Colab notebook window and then clicking the copy link button. Send this link via direct message on Slack ot Dr. Heidi White.

In [ ]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure this API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()

# Print out the url on the portal where we can view the submitted request
print('View this observing request: https://observe.lco.global/requestgroups/{}/'.format(requestgroup_dict['id']))

print('This request was successfully submitted at {}'.format(datetime.datetime.now()))